# Imports

In [14]:
import pandas as pd

In [2]:
from ebay_delivery_prediction_project import Preprocessing, Visualisation, preprocessing_models
Preprocessing.import_test()

Preprocessing successfully imported.


In [3]:
data_object = Preprocessing.read_data(rows_to_read = None, columns = ["b2c_c2c"])
# data_object = Preprocessing.read_data(rows_to_read = None, columns = ["b2c_c2c"])

Reading full data.
Reading ['b2c_c2c'] columns.


In [4]:
print("data_object.keys(): " , data_object.keys())
training_data = data_object["train"]
training_data.columns

data_object.keys():  dict_keys(['train', 'quiz'])


Index(['b2c_c2c'], dtype='object')

# EDA

## b2c_c2c

Annexure : Defines whether the transaction belongs to a business seller; possible values are B2C
(business to consumer) or C2C (consumer to consumer).

In [4]:
training_data["b2c_c2c"].value_counts() # Need to read full data but only certain columns

B2C    10634413
C2C     4365587
Name: b2c_c2c, dtype: int64